In [1]:
import gzip
import shutil
import os

In [1]:
import gzip
import shutil
import os

# Define the directory containing the .gz files
input_dir = 'eebo/'

# Define the directory where you want to save the unzipped files
output_dir = 'unzipped/'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

def unzip_file(input_path, output_path):
    with gzip.open(input_path, 'rb') as f_in:
        with open(output_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

# Iterate over all files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.gz'):
        input_path = os.path.join(input_dir, filename)
        output_filename = filename[:-3]  # Remove the .gz extension
        output_path = os.path.join(output_dir, output_filename)
        
        # Unzip the file
        unzip_file(input_path, output_path)

print(f"All files have been unzipped to {output_dir}")

All files have been unzipped to unzipped/


In [2]:
import os

# Define input and output directories
input_dir = 'unzipped/'
output_dir = 'combined/'
output_file = 'ALL.xml'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Start the combined file with the root XML element
with open(os.path.join(output_dir, output_file), 'w', encoding='utf-8') as outfile:
    outfile.write('<?xml version="1.0" encoding="utf-8"?>\n')
    outfile.write('<ALL>\n')

    # Iterate over each unzipped file
    for filename in os.listdir(input_dir):
        if filename.endswith('.xml'):
            file_path = os.path.join(input_dir, filename)
            
            # Open and read each XML file
            with open(file_path, 'r', encoding='utf-8') as infile:
                content = infile.read()
                
                # Find the start of the <ETS> tag and extract everything until the end
                start_index = content.find('<ETS>')
                end_index = content.rfind('</ETS>') + len('</ETS>')

                if start_index != -1 and end_index != -1:
                    # Write the extracted content to the output file
                    outfile.write(content[start_index:end_index] + '\n')

    # Close the root element
    outfile.write('</ALL>\n')

print(f"All files have been combined into {os.path.join(output_dir, output_file)}")

All files have been combined into combined/ALL.xml


In [27]:
import pandas as pd
import os
from lxml import etree
import re

# Load the CSV file
tcp_df = pd.read_csv('TCP.csv')

# Create a dictionary to store file paths by year
files_by_year = {}

# Ensure the 'date' directory exists
os.makedirs('date', exist_ok=True)

# Iterate over the rows in the DataFrame
for index, row in tcp_df.iterrows():
    file_name = row['TCP']
    date = row['Date']
    
    # Extract the first year from a range like '1658-1664?'
    try:
        # Use a regular expression to find the first year
        match = re.match(r"(\d{4})", date)
        if match:
            year = int(match.group(1))
        else:
            raise ValueError(f"Invalid format: {date}")
    except ValueError:
        print(f"Invalid date format for {file_name}: '{date}', skipping...")
        continue
    
    # Construct the file path
    file_path = f'unzipped/{file_name}.headed.xml'
    
    # Check if the file exists and categorize by year
    if os.path.isfile(file_path):
        if year not in files_by_year:
            files_by_year[year] = []
        files_by_year[year].append(file_path)
    else:
        print(f"File not found: {file_path}")

# Function to combine XML files
def combine_xml_files(file_list, output_file):
    # Create the root element
    root = etree.Element("root")
    
    for file in file_list:
        try:
            # Parse each XML file and append its content
            tree = etree.parse(file)
            root.append(tree.getroot())
        except Exception as e:
            print(f"Error parsing {file}: {e}")
    
    # Write the combined XML to the output file
    tree = etree.ElementTree(root)
    tree.write(output_file, pretty_print=True, xml_declaration=True, encoding='UTF-8')

# Combine files by year
for year, files in files_by_year.items():
    output_file = f'date/{year}.xml'
    combine_xml_files(files, output_file)
    print(f"Combined files for year {year} into {output_file}")

File not found: unzipped/A00005.headed.xml
File not found: unzipped/A00007.headed.xml
File not found: unzipped/A00316.headed.xml
File not found: unzipped/A00363.headed.xml
File not found: unzipped/A00412.headed.xml
File not found: unzipped/A00414.headed.xml
File not found: unzipped/A00419.headed.xml
File not found: unzipped/A00429.headed.xml
File not found: unzipped/A00440.headed.xml
File not found: unzipped/A00525.headed.xml
File not found: unzipped/A00580.headed.xml
File not found: unzipped/A00587.headed.xml
File not found: unzipped/A00597.headed.xml
File not found: unzipped/A00602.headed.xml
File not found: unzipped/A00616.headed.xml
File not found: unzipped/A00617.headed.xml
File not found: unzipped/A00627.headed.xml
File not found: unzipped/A00659.headed.xml
File not found: unzipped/A00698.headed.xml
File not found: unzipped/A00728.headed.xml
File not found: unzipped/A00730.headed.xml
File not found: unzipped/A00753.headed.xml
File not found: unzipped/A00778.headed.xml
File not fo